In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext
!pip install pymorphy2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import fasttext
import fasttext.util

from spacy.lang.ru import Russian
import nltk
from nltk.corpus import stopwords
from string import punctuation

from joblib import dump, load

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
train = pd.read_csv('trainSet.csv')
test = pd.read_csv('testSet.csv')

In [ ]:
y_train = np.array(train['Class'])
y_test = np.array(test['Class'])

In [ ]:
print(np.unique(y_train))
print(np.unique(y_test))

[ 1  3  8 21 22 23 31 32 41 42 43 44]
[ 1  3  8 21 22 23 31 32 41 42 43 44 51 52 53 54 55 56 57 58 59 60 61]


In [ ]:
test = test[test['Class'] <=44]

y_test = np.array(test['Class'])

print(np.unique(y_train))
print(np.unique(y_test))

[ 1  3  8 21 22 23 31 32 41 42 43 44]
[ 1  3  8 21 22 23 31 32 41 42 43 44]


In [ ]:
d = {}
inv_d = {}
l = np.unique(y_train)
for i in range(len(l)):
  d[i] = l[i]
  inv_d[l[i]] = i
print(d)
print(inv_d)
y_train = np.array([inv_d[i] for i in y_train])
y_test = np.array([inv_d[i] for i in y_test])
print(np.unique(y_train))
print(np.unique(y_test))
print(len(y_train))
print(len(y_test))

{0: 1, 1: 3, 2: 8, 3: 21, 4: 22, 5: 23, 6: 31, 7: 32, 8: 41, 9: 42, 10: 43, 11: 44}
{1: 0, 3: 1, 8: 2, 21: 3, 22: 4, 23: 5, 31: 6, 32: 7, 41: 8, 42: 9, 43: 10, 44: 11}
[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11]
2244
963


In [ ]:
X_train = train['Text']
X_test = test['Text']

print(len(X_train))
print(len(X_test))

2244
963


In [ ]:
ft = fasttext.load_model('drive/My Drive/cc.ru.300.bin')

In [ ]:
russian_stopwords = stopwords.words("russian")
nlp = Russian()

def preprocess_text(text):
  tokens = text.lower()
  sentence = nlp(text.lower())
  tokens = [token.text for token in sentence]
  tokens = [token for token in tokens if token not in russian_stopwords\
            and token != " " \
            and token.strip() not in punctuation]
  
  text = " ".join(tokens)
  
  return text

In [ ]:
X_train = np.array([ft.get_sentence_vector(preprocess_text(text)) for text in X_train])
X_test = np.array([ft.get_sentence_vector(preprocess_text(text)) for text in X_test])

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

print(f1_score(y_test, preds, average='macro'))
print(accuracy_score(y_test, preds))
confusion_matrix(y_test, preds)

0.7500703125063323
0.8307372793354102


array([[115,   0,   7,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,  78,  10,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  8,   0,  70,   2,   0,   0,   0,   1,   0,   0,   0,   0],
       [  0,   0,   0,  97,  10,   3,   0,   0,   4,   1,   1,   0],
       [  0,   0,   0,   3,  86,   0,   1,   0,   1,   0,   2,   0],
       [  0,   0,   0,  10,   1,  48,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   2,   1,  66,   7,   0,   0,   0,   0],
       [  0,   1,   0,   0,   0,   0,   1,  81,   0,   0,   0,   0],
       [  0,   0,   0,   1,   1,   0,   0,   0, 103,   1,   3,   0],
       [  1,   0,   0,   3,   0,   1,   0,   0,  27,  13,   0,   0],
       [  0,   0,   0,   0,   5,   0,   0,   0,   9,   1,  43,   0],
       [  0,   0,   0,   0,   0,  12,   0,   0,  17,   2,   0,   0]])

In [ ]:
dump(clf, 'log_reg_fasttext.joblib')

['log_reg_fasttext.joblib']

In [ ]:
clf = SVC().fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

print(f1_score(y_test, preds, average='macro'))
print(accuracy_score(y_test, preds))
confusion_matrix(y_test, preds)

0.9183100934549612
0.9283489096573209


array([[118,   0,   4,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,  83,   5,   0,   0,   0,   0,   1,   0,   0,   0,   0],
       [  4,   4,  72,   1,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 108,   4,   3,   0,   0,   0,   1,   0,   0],
       [  0,   0,   0,   4,  85,   0,   2,   0,   0,   0,   2,   0],
       [  0,   0,   0,   1,   1,  57,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   1,  69,   6,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   1,  82,   0,   0,   0,   0],
       [  0,   0,   0,   1,   0,   0,   0,   0, 108,   0,   0,   0],
       [  0,   0,   1,   1,   0,   0,   0,   0,   7,  35,   0,   1],
       [  0,   0,   0,   0,   3,   0,   0,   0,   1,   0,  54,   0],
       [  0,   0,   0,   0,   0,   6,   0,   0,   2,   0,   0,  23]])

In [ ]:
dump(clf, 'svm_fasttext.joblib')

['svm_fasttext.joblib']

In [ ]:
clf = RandomForestClassifier(n_estimators=200, max_depth=16, random_state=0).fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

print(f1_score(y_test, preds, average='macro'))
print(accuracy_score(y_test, preds))
confusion_matrix(y_test, preds)

0.8282721096113906
0.8733125649013499


array([[115,   0,   7,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,  83,   5,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  4,   1,  75,   1,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  99,   8,   2,   0,   0,   4,   1,   2,   0],
       [  0,   0,   0,   6,  80,   0,   3,   0,   0,   0,   4,   0],
       [  0,   0,   0,   4,   2,  52,   0,   0,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   1,  68,   7,   0,   0,   0,   0],
       [  0,   1,   0,   0,   0,   0,   1,  81,   0,   0,   0,   0],
       [  0,   0,   0,   1,   0,   0,   0,   0, 108,   0,   0,   0],
       [  0,   0,   0,   1,   0,   1,   0,   1,  13,  29,   0,   0],
       [  0,   0,   0,   0,   5,   1,   0,   0,   7,   0,  45,   0],
       [  0,   0,   0,   0,   0,  13,   0,   1,  10,   1,   0,   6]])

In [ ]:
dump(clf, 'rf_fasttext.joblib')

['rf_fasttext.joblib']